# Plain-Text CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import os  # operating system functions
import torch  # torch, for saving trained models
import numpy as np  # handling vectors and matrices
from analysis_functions import (train_tf_model, pred_tf_model,
                                train_pt_model, pred_pt_model)  # own functions
from preproc_functions import list_combine  # own functions
from neural_nets import TFUNET, TFMNET, PTUNET, PTMNET  # own defined neural nets as classes

Using TensorFlow backend.


### datasets

#### univariate training

In [3]:
train_Xu = np.load(dir_data+'train_Xu.npy')
train_yu = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_Xu.shape)
print(train_yu.shape)

(988, 14, 1)
(988, 7)


\>980 Observations for X days for 1 Variable!

#### multivariate training

In [5]:
train_Xm = np.load(dir_data+'train_Xm.npy')
train_ym = np.load(dir_data+'train_ym.npy')

In [6]:
print(train_Xm.shape)
print(train_ym.shape)

(988, 14, 8)
(988, 7)


\>980 Observations for X days for 8 Variables!

#### test set

In [7]:
test_Xu = np.load(dir_data+'test_Xs.npy')[:,:,:1]
test_Xm = np.load(dir_data+'test_Xs.npy')
test_y = np.load(dir_data+'test_ys.npy')

In [8]:
print(test_Xu.shape)
print(test_Xm.shape)
print(test_y.shape)

(59, 14, 1)
(59, 14, 8)
(59, 7)


## Modeling

### Options

In [9]:
# net hyper parameters
# general
lr = 0.001
n_filters = 16
n_linear = 10
# univariate specific
epochs_u = 20
batch_size_u = 4
# multivariate specific
epochs_m = 70
batch_size_m = 16

In [10]:
# dimensions
n_timesteps = train_Xu.shape[1] 
n_features_u = train_Xu.shape[2]
n_features_m = train_Xm.shape[2]
n_outputs = train_yu.shape[1] 

### Tensorflow

#### Univariate

In [11]:
model_tfu = TFUNET(n_features_u, n_filters, n_timesteps, n_linear, n_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
model_tfu, opt_tfu = train_tf_model(train_Xu, train_yu, model_tfu,
                                   batch_size_u, epochs_u, lr)
preds_tfu, rmse_tfu = pred_tf_model(test_Xu, test_y, model_tfu)

Instructions for updating:
Use tf.cast instead.
Overall RMSE for TF model: 592.2436203193602!


#### Multivariate

In [13]:
model_tfm = TFMNET(n_features_m, n_filters, n_timesteps, n_linear, n_outputs)

In [14]:
model_tfm, opt_tfm = train_tf_model(train_Xm, train_ym, model_tfm,
                                    batch_size_u, epochs_u, lr)
preds_tfm, rmse_tfm = pred_tf_model(test_Xm, test_y, model_tfm)

Overall RMSE for TF model: 457.1582536578841!


### Pytorch

#### Univariate

In [15]:
model_ptu = PTUNET(n_features_u, n_filters, n_timesteps, n_linear, n_outputs)

In [16]:
model_ptu, opt_ptu = train_pt_model(train_Xu, train_yu, model_ptu,
                                    batch_size_u, epochs_u, lr)
preds_ptu, rmse_ptu = pred_pt_model(test_Xu, test_y, model_ptu)

Overall RMSE for PT model: 389.8037678459225!


#### Multivariate

In [17]:
model_ptm = PTMNET(n_features_m, n_filters, n_timesteps, n_linear, n_outputs)

In [18]:
model_ptm, opt_ptm = train_pt_model(train_Xm, train_ym, model_ptm,
                                    batch_size_m, epochs_m, lr)
preds_ptm, rmse_ptm = pred_pt_model(test_Xm, test_y, model_ptm)

Overall RMSE for PT model: 418.7436652811684!


### Saving

#### rmse & predictions

In [19]:
l1 = ['rmse', 'preds']
l2 = ['_tf', '_pt']
l3 = ['m', 'u']

In [20]:
file_names = list_combine(list_combine(l1,l2), l3)

In [21]:
for item in file_names[:4]:
    np.save(dir_data+'rmse/'+item, eval(item)[1])

In [22]:
for item in file_names[4:]:
    np.save(dir_data+'predictions/'+item, eval(item)[1])

#### trained model weights

In [23]:
# save in dedicated location
if not os.path.exists(dir_data+'models/'):
    os.makedirs(dir_data+'models/')

In [24]:
# tensorflow
model_tfu.save_weights(dir_data+'models/weights_tfu.h5')
model_tfm.save_weights(dir_data+'models/weights_tfm.h5')

In [25]:
# pytorch
torch.save(model_ptu.state_dict(), dir_data+'models/weights_ptu.pt')
torch.save(model_ptm.state_dict(), dir_data+'models/weights_ptm.pt')